In [1]:
import sys

from PyQt6.QtCore import Qt
from PyQt6.QtWidgets import QApplication, QWidget

class Window(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("test window")
        self.resize(400, 300)  # Optional: set initial size

# Standard boilerplate to run the app
if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = Window()
    window.show()
    sys.exit(app.exec())


SystemExit: 0

c:\Users\HICKMAN\GoDAP_DataScreen\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import sys

from PyQt6.QtCore import Qt
from PySide6.QtCore import Qt
from PyQt6.QtGui import QPainter, QColor
from PyQt6.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout
import time

class TickBar(QWidget):
    def __init__(self, max_value=10, num_ticks=10, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.max_value = max_value
        self.num_ticks = num_ticks
        self.current_value = 0

    def set_value(self, value):
        self.current_value = max(0, min(value, self.max_value))
        self.update()

    def paintEvent(self, event):
        painter = QPainter(self)
        w = self.width()
        h = self.height()
        tick_width = w / self.num_ticks

        active_ticks = int((self.current_value / self.max_value) * self.num_ticks)

        for i in range(self.num_ticks):
            x = i * tick_width
            color = QColor(100, 255, 100) if i < active_ticks else QColor(50, 50, 50)
            painter.setBrush(color)
            painter.drawRect(int(x), 0, int(tick_width - 2), int(h))


class Window(QWidget):
    def __init__(self, max_tickbar_value, num_ticks, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.setWindowTitle("PyQt Driver Dashboard")
        self.resize(1600, 900)
        
        self.max_tickbar_value = max_tickbar_value
        self.num_ticks = num_ticks

        self.initUI(max_tickbar_value, num_ticks)

        

    def initUI(self, max_tickbar_value, num_ticks):
        self.tick_bar = TickBar(self.max_tickbar_value, self.num_ticks)
        self.tick_bar.setFixedSize(300, 400)

        self.label1 = QLabel("Performance_Stat_ICUC")

        self.label2 = QLabel("Exhaust Temp")

        tick_bar_layout = QVBoxLayout()
        tick_bar_layout.addWidget(self.tick_bar)
        tick_bar_layout.addWidget(self.label1)

        self.setLayout(tick_bar_layout)
    
    def set_tickbar_value(self, value):
        self.tick_bar.set_value(value)



    

if __name__ == "__main__":
    app = QApplication(sys.argv)
    dashboard = Window(max_tickbar_value=10, num_ticks=10)
    dashboard.set_tickbar_value(5)  # Set initial value
    dashboard.show()
    time.sleep(5)
    dashboard.set_tickbar_value(1)  # Set initial value
    dashboard.show()
    time.sleep(5)
    dashboard.set_tickbar_value(9)  # Set initial value
    dashboard.show()
    sys.exit(app.exec())


KeyboardInterrupt: 

Now adding in the UDP functionality

In [ ]:
import sys
import math

from PyQt6.QtCore import Qt, QObject, pyqtSignal
from PyQt6.QtGui import QPainter, QColor
from PyQt6.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout
from PyQt6.QtNetwork import QUdpSocket, QHostAddress
import time

class TickBar(QWidget):
    def __init__(self, max_value=10, num_ticks=10, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.max_value = max_value
        self.num_ticks = num_ticks
        self.current_value = 0

    def set_value(self, value):
        self.current_value = max(0, min(value, self.max_value))
        self.update()

    def paintEvent(self, event):
        painter = QPainter(self)
        w = self.width()
        h = self.height()
        tick_width = w / self.num_ticks

        active_ticks = int((self.current_value / self.max_value) * self.num_ticks)

        for i in range(self.num_ticks):
            x = i * tick_width
            color = QColor(100, 255, 100) if i < active_ticks else QColor(50, 50, 50)
            painter.setBrush(color)
            painter.drawRect(int(x), 0, int(tick_width - 2), int(h))


class Window(QWidget):
    def __init__(self, max_tickbar_value, num_ticks, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.setWindowTitle("PyQt Driver Dashboard")
        self.resize(1600, 900)
        
        self.max_tickbar_value = max_tickbar_value
        self.num_ticks = num_ticks

        self.initUI(max_tickbar_value, num_ticks)
        self.receiver = DataReceiver()
        self.receiver.data_recieved.connect(self.set_tickbar_value)
        self.receiver.data_recieved.connect(self.set_tickbar_txt_value)

        

    def initUI(self, max_tickbar_value, num_ticks):
        self.tick_bar = TickBar(self.max_tickbar_value, self.num_ticks)
        self.tick_bar.setFixedSize(600, 800)

        self.label1 = QLabel("AmbAirPress_Cval_PT")

        self.label2 = QLabel("OilTemp_Cval_PT'")
        self.tick_bar_txt = QLabel("n/a")

        tick_bar_layout = QVBoxLayout()
        tick_bar_layout.addWidget(self.tick_bar)
        tick_bar_layout.addWidget(self.tick_bar_txt)
        tick_bar_layout.addWidget(self.label2)

        self.setLayout(tick_bar_layout)
    
    def set_tickbar_value(self, value):
        self.tick_bar.set_value(value)

    def set_tickbar_txt_value(self, value):
        self.tick_bar_txt.setText(f"{value:.2f}")



class DataReceiver(QObject):
    data_recieved = pyqtSignal(float)

    def __init__(self, port=5005):
        super().__init__()
        self.socket = QUdpSocket()
        self.socket.bind(QHostAddress(QHostAddress.SpecialAddress.LocalHost), port)
        self.socket.readyRead.connect(self.read_data)

    def read_data(self):
        while self.socket.hasPendingDatagrams():
            datagram, _, _ = self.socket.readDatagram(self.socket.pendingDatagramSize())
            value = float(datagram.decode())
            self.data_recieved.emit(value)


if __name__ == "__main__":
    app = QApplication(sys.argv)
    dashboard = Window(max_tickbar_value=500, num_ticks=25)
    dashboard.show()
    sys.exit(app.exec())


Trying out a gauge now

In [3]:
from PyQt6.QtWidgets import QApplication, QWidget, QLabel
from PyQt6.QtGui import QPainter, QColor, QBrush, QPen
from PyQt6.QtCore import Qt, QPointF, QRectF, QObject, pyqtSignal
import math
from PyQt6.QtNetwork import QUdpSocket, QHostAddress


class GaugeWidget(QWidget):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.value = 0
        self.min_value = 0
        self.max_value = 100
        self.receiver = DataReceiver()
        self.receiver.data_recieved.connect(self.set_value)


    def set_value(self, value):
        if self.min_value <= value <= self.max_value:
            self.value = value
            self.update() # Trigger a repaint

    def paintEvent(self, event):
        painter = QPainter(self)
        painter.setRenderHint(QPainter.RenderHint.Antialiasing)

        # Draw gauge background (arc)
        rect = QRectF(10, 10, self.width() - 20, self.height() - 20)
        start_angle = 225 * 16 # Start at 225 degrees (clockwise from 3 o'clock)
        span_angle = -270 * 16 # Span 270 degrees counter-clockwise
        painter.setPen(QPen(QColor(100, 100, 100), 5))
        painter.drawArc(rect, start_angle, span_angle)

        # Draw needle
        needle_length = min(self.width(), self.height()) / 2 * 0.8
        center = QPointF(self.width() / 2, self.height() / 2)
        
        # Calculate angle based on value
        normalized_value = (self.value - self.min_value) / (self.max_value - self.min_value)
        angle_rad = math.radians(225 - (normalized_value * 270)) # Adjust for arc direction
        
        needle_end = QPointF(
            center.x() + needle_length * math.cos(angle_rad),
            center.y() - needle_length * math.sin(angle_rad) # Y-axis is inverted in Qt
        )
        
        painter.setPen(QPen(QColor(255, 0, 0), 3))
        painter.drawLine(center, needle_end)

        # Draw center circle
        painter.setBrush(QBrush(QColor(0, 0, 0)))
        painter.drawEllipse(center, 5, 5)

        painter.end()

class DataReceiver(QObject):
    data_recieved = pyqtSignal(float)

    def __init__(self, port=5005):
        super().__init__()
        self.socket = QUdpSocket()
        self.socket.bind(QHostAddress(QHostAddress.SpecialAddress.LocalHost), port)
        self.socket.readyRead.connect(self.read_data)

    def read_data(self):
        while self.socket.hasPendingDatagrams():
            datagram, _, _ = self.socket.readDatagram(self.socket.pendingDatagramSize())
            value = float(datagram.decode())
            self.data_recieved.emit(value)

import sys
app = QApplication(sys.argv)
my_gauge = GaugeWidget()
my_gauge.show()
sys.exit(app.exec())


: 

Putting them together

In [1]:
import sys
import math
import json

from PyQt6.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout, QHBoxLayout
from PyQt6.QtGui import QPainter, QColor, QBrush, QPen, QFont, QFontMetrics
from PyQt6.QtCore import Qt, QPointF, QRectF, QObject, pyqtSignal
from PyQt6.QtNetwork import QUdpSocket, QHostAddress
import time

class TickBar(QWidget):
    def __init__(self, max_value, num_ticks, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.max_value = max_value
        self.num_ticks = num_ticks
        self.current_value = 0
        

    def set_value(self, value):
        self.current_value = max(0, min(value, self.max_value))
        self.update()

    def paintEvent(self, event):
        painter = QPainter(self)
        w = self.width()
        h = self.height()
        tick_width = w / self.num_ticks

        active_ticks = int((self.current_value / self.max_value) * self.num_ticks)

        for i in range(self.num_ticks):
            x = i * tick_width
            color = QColor(100, 255, 100) if i < active_ticks else QColor(50, 50, 50)
            painter.setBrush(color)
            painter.drawRect(int(x), 0, int(tick_width - 2), int(h))

class GaugeWidget(QWidget):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.value = 0
        self.min_value = 0
        self.max_value = 100


    def set_value(self, value):
        if self.min_value <= value <= self.max_value:
            self.value = value
            self.update() # Trigger a repaint

    def paintEvent(self, event):
        painter = QPainter(self)
        painter.setRenderHint(QPainter.RenderHint.Antialiasing)

        # Draw gauge background (arc)
        rect = QRectF(10, 10, self.width() - 20, self.height() - 20)
        start_angle = 225 * 16 # Start at 225 degrees (clockwise from 3 o'clock)
        span_angle = -270 * 16 # Span 270 degrees counter-clockwise
        painter.setPen(QPen(QColor(100, 100, 100), 5))
        painter.drawArc(rect, start_angle, span_angle)

        self.draw_scale_numbers(painter)

        # Draw needle
        needle_length = min(self.width(), self.height()) / 2 * 0.8
        center = QPointF(self.width() / 2, self.height() / 2)
        
        # Calculate angle based on value
        normalized_value = (self.value - self.min_value) / (self.max_value - self.min_value)
        angle_rad = math.radians(225 - (normalized_value * 270)) # Adjust for arc direction
        
        needle_end = QPointF(
            center.x() + needle_length * math.cos(angle_rad),
            center.y() - needle_length * math.sin(angle_rad) # Y-axis is inverted in Qt
        )

        painter.setPen(QPen(QColor(255, 0, 0), 3))
        painter.drawLine(center, needle_end)

        # Draw center circle
        painter.setBrush(QBrush(QColor(0, 0, 0)))
        painter.drawEllipse(center, 5, 5)

        painter.end()
        
    def draw_scale_numbers(self, painter):
        painter.setFont(QFont("Arial", 10)) 
        fm = QFontMetrics(painter.font())

        center = QPointF(self.width() / 2, self.height() / 2)
        radius = min(self.width(), self.height()) / 2 - 30  # Padding from edge
        num_divisions = 10  # Number of labels
        angle_start = 225
        angle_span = 270

        for i in range(num_divisions + 1):
            value = self.min_value + (self.max_value - self.min_value) * i / num_divisions
            text = str(int(value))

            angle_deg = angle_start - (angle_span * i / num_divisions)
            angle_rad = math.radians(angle_deg)

            x = center.x() + radius * math.cos(angle_rad)
            y = center.y() - radius * math.sin(angle_rad)

            text_width = fm.horizontalAdvance(text)
            text_height = fm.height()

            painter.drawText(
                int(x - text_width / 2),
                int(y + text_height / 4),
                text
            )

class LightWidget(QWidget):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.value = 0
        self.min_value = 0
        self.max_value = 1 
        
    def set_value(self, value):
        if self.min_value <= value <= self.max_value:
            self.value = value
            self.update()

    def paintEvent(self, event):
        painter = QPainter(self)
        painter.setRenderHint(QPainter.RenderHint.Antialiasing)
        color = QColor(255, 0, 0) if self.value == 1 else QColor(128, 128, 128)
        painter.setBrush(color)
        painter.drawEllipse(50, 50, 100, 100)


class Window(QWidget):
    def __init__(self, max_tickbar_value, num_ticks, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.setWindowTitle("PyQt Driver Dashboard")
        self.resize(1200, 800)
        
        self.max_tickbar_value = max_tickbar_value
        self.num_ticks = num_ticks

        self.initUI(max_tickbar_value, num_ticks)
        self.receiver = DataReceiver()
        self.receiver.data_recieved.connect(self.route_signal)

    def initUI(self, max_tickbar_value, num_ticks):
# TickBar setup
        self.tick_bar = TickBar(max_tickbar_value, num_ticks)
        self.tick_bar.setFixedSize(200, 50)

        tick_bar_layout = QVBoxLayout()
        tick_bar_layout.setSpacing(0)
        tick_bar_layout.addWidget(self.tick_bar)
        tick_bar_layout.setAlignment(Qt.AlignmentFlag.AlignTop)

        tick_bar_label = QLabel("DPF Temperature")
        tick_bar_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        tick_bar_layout.addWidget(tick_bar_label)
        
        # GaugeWidget setup
        self.gauge = GaugeWidget()
        self.gauge.setFixedSize(250, 250)

        gauge_layout = QVBoxLayout()
        gauge_layout.setSpacing(0)        
        gauge_layout.addWidget(self.gauge)
        gauge_layout.setAlignment(Qt.AlignmentFlag.AlignTop)

        gauge_label = QLabel("Oil Temperature")
        gauge_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        gauge_layout.addWidget(gauge_label)

        self.light = LightWidget()
        self.light.setFixedSize(200, 200)


        light_layout = QVBoxLayout()
        light_layout.setSpacing(0)
        light_layout.addWidget(self.light)
        light_layout.setAlignment(Qt.AlignmentFlag.AlignTop)

        light_label = QLabel("HVCircuitOn Flag")
        light_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        light_layout.addWidget(light_label)


        # Combine both layouts
        main_layout = QHBoxLayout()
        main_layout.addLayout(tick_bar_layout,1)
        main_layout.addLayout(gauge_layout,1)
        main_layout.addLayout(light_layout)

        self.setLayout(main_layout)

    
    def route_signal(self, name, value):
        if name == "EngOilTemp_Cval":
            self.gauge.set_value(value)
        elif name == "DPFTemp_Cval_PT":
            self.tick_bar.set_value(value)
        elif name == "test_flag":
            self.light.set_value(value)



class DataReceiver(QObject):
    data_recieved = pyqtSignal(str, float)

    def __init__(self, port=5005):
        super().__init__()
        self.socket = QUdpSocket()
        self.socket.bind(QHostAddress(QHostAddress.SpecialAddress.LocalHost), port)
        self.socket.readyRead.connect(self.read_data)

    def read_data(self):
        while self.socket.hasPendingDatagrams():
            datagram, _, _ = self.socket.readDatagram(self.socket.pendingDatagramSize())
            try:
                data = datagram.decode()
                parsed = json.loads(data)
                for key, value in parsed.items():
                    self.data_recieved.emit(key, float(value))
            except Exception as e:
                print(f"Failed to parse datagram: {e}")


if __name__ == "__main__":
    app = QApplication(sys.argv)
    dashboard = Window(max_tickbar_value=550, num_ticks=50)
    dashboard.show()
    sys.exit(app.exec())


SystemExit: 0

c:\Users\HICKMAN\driver-dashboard\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Now they need to adapt to any new configuration

In [1]:
import sys
from PyQt6.QtWidgets import QApplication, QWidget, QHBoxLayout, QVBoxLayout, QLabel
from PyQt6.QtCore import Qt, QObject,pyqtSignal, QSocketNotifier
from dashboard_templates import LightWidget, GaugeWidget, TickBar, DataReceiver
from PyQt6.QtNetwork import QUdpSocket, QHostAddress
import json
import socket

class Window(QWidget):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.setWindowTitle("PyQt Driver Dashboard")
        self.resize(1920, 1080)
        

        self.initUI()
        self.receiver = DataReceiver()
        print(f"Receiver created: {self.receiver}")
        self.receiver.data_received.connect(self.route_signal)

    def initUI(self):
        # GaugeWidget setup for Oil Temp 
        self.gauge1 = GaugeWidget(0, 255, "Oil Temp (F)", "EngOilTemp_Cval")
        self.gauge1.setFixedSize(250, 250)

        gauge_layout1 = QVBoxLayout()
        gauge_layout1.setSpacing(0)        
        gauge_layout1.addWidget(self.gauge1)
        gauge_layout1.setAlignment(Qt.AlignmentFlag.AlignTop)
        
        # GaugeWidget setup for Coolant temp
        self.gauge2 = GaugeWidget(0, 255, "Coolant Temp (F)", "EngCoolTemp_Cval")
        self.gauge2.setFixedSize(250, 250)

        gauge_layout2 = QVBoxLayout()
        gauge_layout2.setSpacing(0)        
        gauge_layout2.addWidget(self.gauge2)
        gauge_layout2.setAlignment(Qt.AlignmentFlag.AlignTop)


        # GaugeWidget setup for Oil pressure
        self.gauge3 = GaugeWidget(10, 80, "Oil Pressure (psi)", "EngOilPress_Cval_CPC")
        self.gauge3.setFixedSize(250, 250)

        gauge_layout3 = QVBoxLayout()
        gauge_layout3.setSpacing(0)        
        gauge_layout3.addWidget(self.gauge3)
        gauge_layout3.setAlignment(Qt.AlignmentFlag.AlignTop)

        
        # TickBar setup for system voltage
        self.tickbar1 = TickBar(-14.4, 14.4, 30, "System Voltage","SysVolt_Stat")
        self.tickbar1.setFixedSize(250,60)

        tickbar1_layout = QVBoxLayout()
        tickbar1_layout.setSpacing(0)
        tickbar1_layout.addWidget(self.tickbar1)
        tickbar1_layout.setAlignment(Qt.AlignmentFlag.AlignTop)

        # LightWidget for Eng. Protection Shutdown Engine flag
        self.light1 = LightWidget("Eng. Prot. Shtdn. Flag", "EngProtShtdnEng_Stat_CPC")
        self.light1.setFixedSize(200, 200)

        light_layout1 = QVBoxLayout()
        light_layout1.setSpacing(0)
        light_layout1.addWidget(self.light1)
        light_layout1.setAlignment(Qt.AlignmentFlag.AlignTop)


        # Combine both layouts
        main_layout1 = QHBoxLayout()
        main_layout1.addLayout(gauge_layout1)
        main_layout1.addLayout(gauge_layout2)
        main_layout1.addLayout(gauge_layout3)
        
        main_layout2 = QHBoxLayout()
        #main_layout2.addLayout(light_layout1)
        main_layout2.addLayout(tickbar1_layout)

        window_layout = QVBoxLayout()
        window_layout.addLayout(main_layout1)
        window_layout.addLayout(main_layout2)

        self.setLayout(window_layout)

    
    def route_signal(self, name, value):
        self.gauge1.set_value(value,name)
        self.gauge2.set_value(value,name)
        self.gauge3.set_value(value,name)
        self.tickbar1.set_value(value,name)
        self.light1.set_value(value,name)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    dashboard = Window()
    dashboard.show()
    sys.exit(app.exec())


[DataReceiver] Bound to 6000
Receiver created: <dashboard_templates.DataReceiver object at 0x000001617B0C1C60>


SystemExit: 0

c:\Users\HICKMAN\driver-dashboard\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


We need to parameterize it all now, so that users can easily hot swap dashboard templates

In [ ]:
import sys
from PyQt6.QtWidgets import QApplication, QWidget, QHBoxLayout, QVBoxLayout, QLabel
from PyQt6.QtCore import Qt, QObject,pyqtSignal, QSocketNotifier
from dashboard_templates import DataReceiver, AddToWindow
from PyQt6.QtNetwork import QUdpSocket, QHostAddress
import json
import socket

class Window(QWidget):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.setWindowTitle("PyQt Driver Dashboard")
        self.resize(1920, 1080)
        self.allwidgets = {}
        

        self.initUI()
        self.receiver = DataReceiver()
        print(f"Receiver created: {self.receiver}")
        self.receiver.data_received.connect(self.route_signal)

    def initUI(self):
        mainlayout = QVBoxLayout()

        # (widgetname, signalname, label, minvalue (not needed for light), maxvalue (not needed for light), num of ticks (only needed for tickbar))
        row1 = [("Gauge", "RPM", "RPM", 0, 2200, None),
                ("Gauge", "OilPress", "Oil Pressure (psi)", 0, 100, None),]
        mainlayout.addLayout(self.widget_row(row1))
        row2 = [("Light", "RPM_Above_1700", "HighRPMFlag", 0, 1, None),
                ("Tickbar", "BatteryVoltage", "BatVolt", 10, 14.4, 20)]
        mainlayout.addLayout(self.widget_row(row2))
        self.setLayout(mainlayout)

    def widget_row(self, widgets):
        row_layout = QHBoxLayout()
        for widget in widgets:
            widget_creator = AddToWindow(self, *widget)
            widget_name = f"{widget[0]}_{widget[1]}"  
            row_layout.addWidget(self.allwidgets[widget_name])
        return row_layout

    def route_signal(self, name, value):
        for key, a in self.allwidgets.items():
            self.allwidgets[key].set_value(value,name)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    dashboard = Window()
    dashboard.show()
    sys.exit(app.exec())

[DataReceiver] Bound to 6000
Receiver created: <dashboard_templates.DataReceiver object at 0x0000028A8178D750>


KeyboardInterrupt: 

In [1]:
from PyQt6.QtWidgets import (
    QApplication, QWidget, QPushButton, QVBoxLayout, QDialog, QLabel,
    QComboBox, QLineEdit, QDialogButtonBox, QGridLayout, QHBoxLayout
)
from PyQt6.QtCore import Qt
from dashboard_templates import DataReceiver, AddToWindow

# Config dialog
class AddWidgetDialog(QDialog):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Add Widget")
        layout = QVBoxLayout()

        self.type_box = QComboBox()
        self.type_box.addItems(["Gauge", "Light", "Tickbar"])

        self.config_input1 = QLineEdit()
        self.config_input1.setPlaceholderText("Enter the signal name that you want to show on this widget::")

        self.config_input2 = QLineEdit()
        self.config_input2.setPlaceholderText("Enter the label for this widget::")

        self.config_input3 = QLineEdit()
        self.config_input3.setPlaceholderText("Enter the maximum value for this widget::")

        self.config_input4 = QLineEdit()
        self.config_input4.setPlaceholderText("Enter the minimum value for this widget::")

        self.config_input5 = QLineEdit()
        self.config_input5.setPlaceholderText("Enter the number of ticks if you selected a tickbar widget::")


        layout.addWidget(QLabel("Choose Widget Type:"))
        layout.addWidget(self.type_box)
        layout.addWidget(QLabel("Configuration:"))
        layout.addWidget(self.config_input1)
        layout.addWidget(self.config_input2)
        layout.addWidget(self.config_input3)
        layout.addWidget(self.config_input4)
        layout.addWidget(self.config_input5)

        buttons = QDialogButtonBox.StandardButton.Ok | QDialogButtonBox.StandardButton.Cancel
        self.buttonBox = QDialogButtonBox(buttons)
        self.buttonBox.accepted.connect(self.accept)
        self.buttonBox.rejected.connect(self.reject)

        layout.addWidget(self.buttonBox)
        self.setLayout(layout)

    def getData(self):
        return {"type" : self.type_box.currentText(), 
                "signal" : self.config_input1.text(), 
                "label" : self.config_input2.text(), 
                "max": float(self.config_input3.text()), 
                "min" : float(self.config_input4.text()), 
                "ticks" : int(self.config_input5.text())}

# Each slot in the layout
class SlotWidget(QWidget):
    def __init__(self, parent_layout, all_widgets_dict):
        super().__init__()
        self.parent_layout = parent_layout
        self.layout = QVBoxLayout()
        self.setLayout(self.layout)
        self.allwidgets = all_widgets_dict

        self.add_button = QPushButton("Add Widget +")
        self.add_button.clicked.connect(self.openAddDialog)
        self.layout.addWidget(self.add_button)

    def openAddDialog(self):
        dialog = AddWidgetDialog()
        if dialog.exec():
            data = dialog.getData()
            widget_type = data["type"]
            self.add_button.deleteLater()
            if widget_type == "Gauge":
                row = [("Gauge", data["signal"], data["label"], data["min"], data["max"], None)]
            elif widget_type == "Light":
                row = [("Light", data["signal"], data["label"], None, None, None)]
            elif widget_type == "Tickbar":
                row = [("Tickbar", data["signal"], data["label"], data["min"], data["max"], data["ticks"])]
            self.layout.addLayout(self.widget_row(row))

    def widget_row(self, widgets):
        row_layout = QHBoxLayout()
        for widget in widgets:
            widget_creator = AddToWindow(self, *widget, remove_callback=self.restore_add_button)
            widget_name = f"{widget[0]}_{widget[1]}"  
            row_layout.addWidget(widget_creator.container, alignment=Qt.AlignmentFlag.AlignHCenter)
        return row_layout
    
    def restore_add_button(self):
        self.add_button = QPushButton("Add Widget +")
        self.add_button.clicked.connect(self.openAddDialog)
        self.layout.addWidget(self.add_button)


class Window(QWidget):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.setWindowTitle("PyQt Driver Dashboard")
        self.resize(1920, 1080)
        layout = QGridLayout()
        self.allwidgets = {}

        positions = [(i, j) for i in range(2) for j in range(3)]
        for pos in positions:
            slot = SlotWidget(layout, self.allwidgets)
            layout.addWidget(slot, *pos)

        self.setLayout(layout)
        self.receiver = DataReceiver()
        print(f"Receiver created: {self.receiver}")
        self.receiver.data_received.connect(self.route_signal)


    def route_signal(self, name, value):
        for key, a in self.allwidgets.items():
            self.allwidgets[key].set_value(value,name)

if __name__ == "__main__":
    app = QApplication([])
    window = Window()
    window.show()
    app.exec()


[DataReceiver] Bound to 6000
Receiver created: <dashboard_templates.DataReceiver object at 0x0000020D070F9FC0>


In [2]:
from PyQt6.QtWidgets import (
    QApplication, QWidget, QPushButton, QVBoxLayout, QDialog, QLabel,
    QComboBox, QLineEdit, QDialogButtonBox, QGridLayout, QHBoxLayout
)
from PyQt6.QtCore import Qt
from dashboard_templates import DataReceiver, AddToWindow
import pickle
import os

# popup window created after pressing Add Widget button
class AddWidgetDialog(QDialog):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Add Widget")
        layout = QVBoxLayout()

        # widget dropdown and text boxes
        self.type_box = QComboBox()
        self.type_box.addItems(["Gauge", "Light", "Tickbar"])

        self.config_input1 = QLineEdit()
        self.config_input1.setPlaceholderText("Ex: EngineOilTemp_Cval")

        self.config_input2 = QLineEdit()
        self.config_input2.setPlaceholderText("Ex: Engine Oil Temp (F)")

        self.config_input3 = QLineEdit()
        self.config_input3.setPlaceholderText("Ex: 255")

        self.config_input4 = QLineEdit()
        self.config_input4.setPlaceholderText("Ex: 0")

        self.config_input5 = QLineEdit()
        self.config_input5.setPlaceholderText("Ex: 0, if not using tickbar, Ex: 10, if using tickbar")


        # adding widgets to popup window layout
        layout.addWidget(QLabel("Choose Widget Type:"))
        layout.addWidget(self.type_box)
        layout.addWidget(QLabel("Enter the signal name that you want to display on this widget::"))
        layout.addWidget(self.config_input1)
        layout.addWidget(QLabel("Enter the label for this widget::"))
        layout.addWidget(self.config_input2)
        layout.addWidget(QLabel("Enter the maximum value for this widget::"))
        layout.addWidget(self.config_input3)
        layout.addWidget(QLabel("Enter the minimum value for this widget::"))
        layout.addWidget(self.config_input4)
        layout.addWidget(QLabel("Enter the number of ticks if you selected a tickbar widget (put 0 if not using a tickbar)::"))
        layout.addWidget(self.config_input5)

        # Okay and cancel buttons to continue/close window
        buttons = QDialogButtonBox.StandardButton.Ok | QDialogButtonBox.StandardButton.Cancel
        self.buttonBox = QDialogButtonBox(buttons)
        self.buttonBox.accepted.connect(self.accept)
        self.buttonBox.rejected.connect(self.reject)

        layout.addWidget(self.buttonBox)
        self.setLayout(layout)

    # return function for user inputs
    def getData(self):
        return {"type" : self.type_box.currentText(), 
                "signal" : self.config_input1.text(), 
                "label" : self.config_input2.text(), 
                "max": float(self.config_input3.text()), 
                "min" : float(self.config_input4.text()), 
                "ticks" : float(self.config_input5.text())}
    
class SaveConfigDialog(QDialog):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Save Dashboard Configuration")
        layout = QVBoxLayout()

        layout.addWidget(QLabel("Enter a name for your configuration file:: "))
        self.save_input1 = QLineEdit()
        self.save_input1.setPlaceholderText("Ex: Noah_RegenTestingDashboard")
        layout.addWidget(self.save_input1)


        # Okay and cancel buttons to continue/close window
        buttons = QDialogButtonBox.StandardButton.Ok | QDialogButtonBox.StandardButton.Cancel
        self.buttonBox = QDialogButtonBox(buttons)
        self.buttonBox.accepted.connect(self.accept)
        self.buttonBox.rejected.connect(self.reject)

        layout.addWidget(self.buttonBox)
        self.setLayout(layout)
        
    # return function for user inputs
    def getData(self):
        return self.save_input1.text()
    
class LoadConfigDialog(QDialog):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Load Dashboard Configuration")
        layout = QVBoxLayout()

        layout.addWidget(QLabel("Please select the configuration to load:: "))
        self.load_config_box = QComboBox()
        loadable_configs = [item for item in os.listdir(r"F:\driver_dashboard_configs")]
        self.load_config_box.addItems(loadable_configs)

        layout.addWidget(self.load_config_box)

        # Okay and cancel buttons to continue/close window
        buttons = QDialogButtonBox.StandardButton.Ok | QDialogButtonBox.StandardButton.Cancel
        self.buttonBox = QDialogButtonBox(buttons)
        self.buttonBox.accepted.connect(self.accept)
        self.buttonBox.rejected.connect(self.reject)


        layout.addWidget(self.buttonBox)
        self.setLayout(layout)
        
    # return function for user inputs
    def getData(self):
        return self.load_config_box.currentText()



# instance creator for the different add widget buttons on initial dashboard window
class SlotWidget(QWidget):
    def __init__(self, parent_layout, all_widgets_dict, i, j):
        super().__init__()
        self.parent_layout = parent_layout
        self.layout = QVBoxLayout()
        self.setLayout(self.layout)
        self.allwidgets = all_widgets_dict
        self.i = i
        self.j = j 

        self.add_button = QPushButton("Add Widget +")
        self.add_button.clicked.connect(self.openAddDialog)
        self.layout.addWidget(self.add_button)

    # Function to open the widget adding pop-up window
    def openAddDialog(self):
        dialog = AddWidgetDialog()
        if dialog.exec():
            data = dialog.getData()
            widget_type = data["type"]
            self.add_button.deleteLater()
            if widget_type == "Gauge":
                row = [("Gauge", data["signal"], data["label"], data["min"], data["max"], None, self.i, self.j)]
            elif widget_type == "Light":
                row = [("Light", data["signal"], data["label"], None, None, None, self.i, self.j)]
            elif widget_type == "Tickbar":
                row = [("Tickbar", data["signal"], data["label"], data["min"], data["max"], data["ticks"], self.i, self.j)]
            self.layout.addLayout(self.widget_row(row))

    # Function to create the selected widget and add it to the window
    def widget_row(self, widgets):
        row_layout = QHBoxLayout()
        for widget in widgets:
            widget_creator = AddToWindow(self, *widget, remove_callback=self.restore_add_button)
            widget_name = f"{widget[0]}_{widget[1]}"  
            row_layout.addWidget(widget_creator.container, alignment=Qt.AlignmentFlag.AlignHCenter)
        return row_layout
    
    # Callback function that recreates add widget button when a widget is removed
    def restore_add_button(self):
        self.add_button = QPushButton("Add Widget +")
        self.add_button.clicked.connect(self.openAddDialog)
        self.layout.addWidget(self.add_button)

class LoadPreset(QWidget):
    def __init__(self, parent_layout, widgetname, signalname, label, minvalue, maxvalue, numticks, i, j):
        super().__init__()
        self.parent_layout = parent_layout
        self.layout = QVBoxLayout()
        self.setLayout(self.layout)
        self.widgetname = widgetname
        self.signalname = signalname
        self.label = label
        self.min_value = minvalue
        self.max_value = maxvalue
        self.numticks = numticks
        self.i = i
        self.j = j 

        if self.widgetname == "Gauge":
            row = [("Gauge", self.widgetname, self.label, self.min_value, self.max_value, None, self.i, self.j)]
        elif self.widgetname == "Light":
            row = [("Light", self.widgetname, self.label, None, None, None, self.i, self.j)]
        elif self.widgetname == "Tickbar":
            row = [("Tickbar", self.widgetname, self.label, self.min_value, self.max_value, self.numticks, self.i, self.j)]
        self.layout.addLayout(self.LoadWidgetPreset(row))

    def LoadPresetWidget(self, widgets):
        row_layout = QHBoxLayout()
        for widget in widgets:
            widget_creator = AddToWindow(self, *widget, remove_callback=self.restore_add_button)
            widget_name = f"{widget[0]}_{widget[1]}"  
            row_layout.addWidget(widget_creator.container, alignment=Qt.AlignmentFlag.AlignHCenter)
        return row_layout


class Window(QWidget):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.setWindowTitle("PyQt Driver Dashboard")
        self.resize(1920, 1080)
        mainwindowlayout = QVBoxLayout()
        layout = QGridLayout()
        self.allwidgets = {}

        # creating the add widget buttons
        positions = [(i, j) for i in range(2) for j in range(3)]
        for pos in positions:
            i,j  = pos
            slot = SlotWidget(layout, self.allwidgets, i, j)
            layout.addWidget(slot, *pos)
        
        mainwindowlayout.addLayout(layout)

        saveloadbutton_layout = QHBoxLayout()
        self.save_button = QPushButton("Save Dashboard Config")
        self.save_button.clicked.connect(self.openSaveDialog)
        saveloadbutton_layout.addWidget(self.save_button)

        self.save_button = QPushButton("Load Dashboard Config")
        self.save_button.clicked.connect(self.openLoadDialog)
        saveloadbutton_layout.addWidget(self.save_button)

        mainwindowlayout.addLayout(saveloadbutton_layout)


        self.setLayout(mainwindowlayout)
        self.receiver = DataReceiver()
        print(f"Receiver created: {self.receiver}")
        self.receiver.data_received.connect(self.route_signal)

    def openSaveDialog(self):
        dialog = SaveConfigDialog()
        if dialog.exec():
            data = dialog.getData()
            print(self.allwidgets)
            self.writepickle(data, self.allwidgets)

    def openLoadDialog(self):
        dialog = LoadConfigDialog()
        if dialog.exec():
            filepath = dialog.getData()
            loaddata = self.readpickle(filepath)
            print(loaddata)

            loadedlayout = QVBoxLayout()
            all_positions = [(i,j) for i in range(2) for j in range(3)]
            positions = [i for i in loaddata["placement"]]
            for pos in positions:
                if not all_positions.find(positions):
                    i,j  = pos
                    slot = SlotWidget(loadedlayout, self.allwidgets, i, j)
                    loadedlayout.addWidget(slot, *pos)
                else: 
                    i,j = pos
                    slot = LoadPreset()
                    loadedlayout.addWidget(slot,*pos)

            self.setLayout(loadedlayout)


            

    def writepickle(self, config, savedata):
        filename = f"F:\driver_dashboard_configs\{config}.pkl"
        with open(filename, 'wb') as file:
            pickle.dump(savedata,file)
        print("New .pkl files have been written.")

    def readpickle(self, loadfile):
        filename = f"F:\driver_dashboard_configs\{loadfile}"
        with open(filename, 'rb') as file:
            loaddata = pickle.load(file)
        print(".pkl files have been read.")
        return loaddata

    # routing each incoming UDP signal to its corresponding widget
    def route_signal(self, name, value):
        for key, a in self.allwidgets.items():
            self.allwidgets[key].set_value(value,name)

# running code
if __name__ == "__main__":
    app = QApplication([])
    window = Window()
    window.show()
    app.exec()


[DataReceiver] Bound to 6000
Receiver created: <dashboard_templates.DataReceiver object at 0x000001C12BD76B00>
.pkl files have been read.
{'Gauge_RPM': {'widget_type': 'Gauge', 'signal': 'RPM', 'label': 'RPM', 'minvalue': '0.0', 'maxvalue': '2000.0', 'numticks': 'None', 'placement': (0, 1)}}


KeyError: 'placement'

KeyboardInterrupt: 